# Step 1: Initialization

In [283]:
import numpy as np
import pandas as pd

#create dataframe from flight data csv
flights_raw = pd.read_csv(r"C:\Users\mm183\Documents\flight_data\flight_data.csv")

#explore data
print(flights_raw.shape)
print(flights_raw.dtypes)
print(flights_raw.head())
print(flights_raw.tail())

(1856061, 43)
QUARTER                    int64
MONTH                      int64
DAY_OF_MONTH               int64
DAY_OF_WEEK                int64
FL_DATE                   object
UNIQUE_CARRIER            object
FL_NUM                     int64
ORIGIN_AIRPORT_ID          int64
ORIGIN_AIRPORT_SEQ_ID      int64
ORIGIN_CITY_MARKET_ID      int64
ORIGIN                    object
ORIGIN_CITY_NAME          object
ORIGIN_STATE_ABR          object
ORIGIN_STATE_NM           object
DEST_AIRPORT_ID            int64
DEST_AIRPORT_SEQ_ID        int64
DEST_CITY_MARKET_ID        int64
DEST                      object
DEST_CITY_NAME            object
DEST_STATE_ABR            object
DEST_STATE_NM             object
CRS_DEP_TIME               int64
DEP_TIME                 float64
DEP_DELAY                float64
DEP_DELAY_NEW            float64
WHEELS_ON                float64
TAXI_IN                  float64
CRS_ARR_TIME               int64
ARR_TIME                 float64
ARR_DELAY                floa

# Step 2: Make data more palatable 
  - reformat headers and reindex
  - address missing values

In [284]:
#make column names lowercase
flights_raw.columns = map(str.lower, flights_raw.columns)

#view percentage of columns consisting of NaN
print(flights_raw.isnull().mean())

quarter                  0.000000
month                    0.000000
day_of_month             0.000000
day_of_week              0.000000
fl_date                  0.000000
unique_carrier           0.000000
fl_num                   0.000000
origin_airport_id        0.000000
origin_airport_seq_id    0.000000
origin_city_market_id    0.000000
origin                   0.000000
origin_city_name         0.000000
origin_state_abr         0.000000
origin_state_nm          0.000000
dest_airport_id          0.000000
dest_airport_seq_id      0.000000
dest_city_market_id      0.000000
dest                     0.000000
dest_city_name           0.000000
dest_state_abr           0.000000
dest_state_nm            0.000000
crs_dep_time             0.000000
dep_time                 0.014290
dep_delay                0.014290
dep_delay_new            0.014290
wheels_on                0.015120
taxi_in                  0.015120
crs_arr_time             0.000000
arr_time                 0.015120
arr_delay     

In [285]:
'''reindex and sort dataframe using descriptive columns without missing values
    - combinations of flight date, origin, destination, carrier, and flight number correspond to individual flights
'''
flights = flights_raw.set_index(['fl_date','origin','dest','unique_carrier','fl_num']).sort_index()

#verify index values are unique i.e. there are no duplicate flights
assert len(flights.index) == len(flights.index.unique()), "Index values are not unique"

In [286]:
#make lists of columns with high proportions of missing values and columns with low proportions of missing values
high_per_nan = [col for col in flights if flights[col].isnull().mean() > .5 ]
low_per_nan = [col for col in flights if 0 < flights[col].isnull().mean() < .5]

#browse dataframes with high and low proportions of missing values
print('high_per_nan', flights[high_per_nan].head())
print('low_per_nan', flights[low_per_nan].head())

high_per_nan                                               carrier_delay  weather_delay  \
fl_date    origin dest unique_carrier fl_num                                 
2016-01-01 ABE    ATL  EV             5197              NaN            NaN   
                                      5579             22.0            0.0   
                  ORD  EV             6021             21.0            0.0   
           ABQ    ATL  DL             1784              NaN            NaN   
                                      2007              NaN            NaN   

                                              nas_delay  security_delay  \
fl_date    origin dest unique_carrier fl_num                              
2016-01-01 ABE    ATL  EV             5197          NaN             NaN   
                                      5579          0.0             0.0   
                  ORD  EV             6021          0.0             0.0   
           ABQ    ATL  DL             1784          NaN          

In [287]:
'''set aside columns containing high percentage of missing values
    - such columns contain circumstantial flight delay information for a small proportion of total flights
'''
#create dataframe containing high percentage NaN columns
hpn_cols = flights[high_per_nan]

#drop columns from flights dataframe
flights = flights.drop(columns=high_per_nan)

In [288]:
'''explore columns containing low percentage of missing values
    - create dataframe of columns with low percentage of NaN values
    - view descriptive statistics
    - view standard deviation of columns as percentage of mean to gauge variability
    - view percentage of total rows containing missing values
'''
lpn_cols = flights[low_per_nan]
print(lpn_cols.describe(include='all'))
print('coefficient of variation:','\n', lpn_cols.apply(lambda x: (x.std() / x.mean()) * 100))
print('percent of rows containing nan:', (len(flights[flights.isnull().any(axis=1)]) / len(flights)) * 100)

           dep_time     dep_delay  dep_delay_new     wheels_on       taxi_in  \
count  1.829537e+06  1.829537e+06   1.829537e+06  1.827997e+06  1.827997e+06   
mean   1.334419e+03  9.812064e+00   1.267238e+01  1.471640e+03  7.562686e+00   
std    5.004642e+02  4.166410e+01   4.064121e+01  5.270121e+02  6.047238e+00   
min    1.000000e+00 -6.000000e+01   0.000000e+00  1.000000e+00  1.000000e+00   
25%    9.200000e+02 -5.000000e+00   0.000000e+00  1.053000e+03  4.000000e+00   
50%    1.329000e+03 -2.000000e+00   0.000000e+00  1.510000e+03  6.000000e+00   
75%    1.741000e+03  7.000000e+00   7.000000e+00  1.914000e+03  9.000000e+00   
max    2.400000e+03  2.040000e+03   2.040000e+03  2.400000e+03  2.500000e+02   

           arr_time     arr_delay  arr_delay_new  crs_elapsed_time  \
count  1.827997e+06  1.824403e+06   1.824403e+06      1.856058e+06   
mean   1.476238e+03  4.206204e+00   1.259892e+01      1.463799e+02   
std    5.314493e+02  4.389959e+01   4.031154e+01      7.673182e+01   

In [289]:
'''set aside rows containing missing values from flights dataframe
    - due to high variability in critical columns (pertaining to delay times) imputation of means would not be appropriate
    - rows constitute small percentage of total data
    - still possible at a later point to calculate missing values from remaining delay information
'''
#create dataframe of rows with null values
nan_rows = flights[flights.isnull().any(axis=1)]

#drop rows with null values from flights
flights = flights.drop(nan_rows.index)

#view percentage of remaining columns consisting of NaN 
print(flights.isnull().mean())

#view shape of resulting flights dataframe
print(flights.shape)

quarter                  0.0
month                    0.0
day_of_month             0.0
day_of_week              0.0
origin_airport_id        0.0
origin_airport_seq_id    0.0
origin_city_market_id    0.0
origin_city_name         0.0
origin_state_abr         0.0
origin_state_nm          0.0
dest_airport_id          0.0
dest_airport_seq_id      0.0
dest_city_market_id      0.0
dest_city_name           0.0
dest_state_abr           0.0
dest_state_nm            0.0
crs_dep_time             0.0
dep_time                 0.0
dep_delay                0.0
dep_delay_new            0.0
wheels_on                0.0
taxi_in                  0.0
crs_arr_time             0.0
arr_time                 0.0
arr_delay                0.0
arr_delay_new            0.0
crs_elapsed_time         0.0
actual_elapsed_time      0.0
air_time                 0.0
flights                  0.0
distance                 0.0
dtype: float64
(1824403, 31)


# Step 3: Evaluate remaining data
  - reshape if necessary
  - drop redundant columns

In [290]:
#view dtypes for remaining columns
flights.dtypes

quarter                    int64
month                      int64
day_of_month               int64
day_of_week                int64
origin_airport_id          int64
origin_airport_seq_id      int64
origin_city_market_id      int64
origin_city_name          object
origin_state_abr          object
origin_state_nm           object
dest_airport_id            int64
dest_airport_seq_id        int64
dest_city_market_id        int64
dest_city_name            object
dest_state_abr            object
dest_state_nm             object
crs_dep_time               int64
dep_time                 float64
dep_delay                float64
dep_delay_new            float64
wheels_on                float64
taxi_in                  float64
crs_arr_time               int64
arr_time                 float64
arr_delay                float64
arr_delay_new            float64
crs_elapsed_time         float64
actual_elapsed_time      float64
air_time                 float64
flights                  float64
distance  

In [291]:
'''separate columns based on component of flight information
    - date
    - location (origin & destination)
    - status timestamps and other metrics
'''
#create dataframes for groups of columns
date_info = flights.loc[:, 'quarter':'day_of_week']
location_info = flights.loc[:, 'origin_airport_id':'dest_state_nm']
flight_metrics = flights.loc[:, 'crs_dep_time':'distance']

In [292]:
'''define function for creating dictionary with unique values in columns in given dataframe
    - key as column name
    - value as array of unique values in column 
    - sort array for readability
'''
def uv_dict(dataframe):
    unique_col_values = {col: array for (col, array) in [(col, dataframe[col].unique()) for col in dataframe]}
    for col, array in unique_col_values.items():
        array = array.sort()
    return unique_col_values

### Date 

In [293]:
#view unique values in flight date columns
print(uv_dict(date_info))

#view columns pertaining to flight date
date_info.head()

{'quarter': array([1, 3, 4], dtype=int64), 'month': array([ 1,  8, 11, 12], dtype=int64), 'day_of_month': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
      dtype=int64), 'day_of_week': array([1, 2, 3, 4, 5, 6, 7], dtype=int64)}


quarter  month  day_of_month  \
fl_date    origin dest unique_carrier fl_num                                 
2016-01-01 ABE    ATL  EV             5197          1      1             1   
                                      5579          1      1             1   
                  ORD  EV             6021          1      1             1   
           ABQ    ATL  DL             1784          1      1             1   
                                      2007          1      1             1   

                                              day_of_week  
fl_date    origin dest unique_carrier fl_num               
2016-01-01 ABE    ATL  EV             5197              5  
                                      5579              5  
                  ORD  EV             6021              5  
           ABQ    ATL  DL             1784              5  
                                      2007              5

Keep all columns containing flight date information in flights dataframe
- may prove useful in further analysis/storytelling

## Location

### Bureau of Transportation Statistics Definitions
(https://www.transtats.bts.gov/Fields.asp?Table_ID=292)

OriginAirportID 
    - An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.	
OriginAirportSeqID	
    - An identification number assigned by US DOT to identify a unique airport at a given point of time. Airport attributes, such as airport name or coordinates, may change over time.	
OriginCityMarketID	
    - City Market ID is an identification number assigned by US DOT to identify a city market. Use this field to consolidate airports serving the same city market.	
Origin	
    - Origin Airport		
OriginCityName
    - Origin City		 
OriginStateAbr	
    - Origin State abbreviation		
OriginStateName	
    - Origin Airport, State Name
DestAirportID 
    - An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.	
DestAirportSeqID	
    - An identification number assigned by US DOT to identify a unique airport at a given point of time. Airport attributes, such as airport name or coordinates, may change over time.	
DestCityMarketID	
    - City Market ID is an identification number assigned by US DOT to identify a city market. Use this field to consolidate airports serving the same city market.	
Dest	
    - Destination Airport		
DestCityName
    - Destination City		 
DestStateAbr	
    - Destination State abbreviation		
DestStateName	
    - Destination Airport, State Name

In [294]:
#create dictionary of unique values in location columns
location_uv = uv_dict(location_info)
print(location_uv)

#view columns pertaining to flight location
location_info.head()

{'origin_airport_id': array([10135, 10136, 10140, 10141, 10146, 10154, 10155, 10157, 10158,
       10165, 10170, 10185, 10208, 10245, 10257, 10279, 10299, 10333,
       10372, 10397, 10408, 10423, 10431, 10434, 10469, 10529, 10551,
       10561, 10577, 10581, 10599, 10620, 10627, 10631, 10666, 10685,
       10693, 10713, 10721, 10728, 10731, 10732, 10739, 10747, 10754,
       10779, 10781, 10785, 10792, 10800, 10821, 10849, 10868, 10874,
       10918, 10926, 10980, 10990, 10994, 11003, 11013, 11042, 11049,
       11057, 11066, 11076, 11097, 11109, 11122, 11140, 11146, 11150,
       11193, 11203, 11252, 11259, 11267, 11278, 11292, 11298, 11308,
       11336, 11337, 11413, 11423, 11433, 11447, 11471, 11481, 11495,
       11503, 11525, 11537, 11540, 11577, 11587, 11603, 11612, 11617,
       11618, 11624, 11630, 11637, 11638, 11641, 11648, 11695, 11697,
       11721, 11775, 11778, 11823, 11865, 11867, 11884, 11898, 11905,
       11921, 11953, 11973, 11977, 11980, 11982, 11986, 11995, 11996

origin_airport_id  \
fl_date    origin dest unique_carrier fl_num                      
2016-01-01 ABE    ATL  EV             5197                10135   
                                      5579                10135   
                  ORD  EV             6021                10135   
           ABQ    ATL  DL             1784                10140   
                                      2007                10140   

                                              origin_airport_seq_id  \
fl_date    origin dest unique_carrier fl_num                          
2016-01-01 ABE    ATL  EV             5197                  1013503   
                                      5579                  1013503   
                  ORD  EV             6021                  1013503   
           ABQ    ATL  DL             1784                  1014003   
                                      2007                  1014003   

                                              origin_city_market_id  \
fl_date    origin dest unique_carrier fl_num                          
2016-01-01 ABE    ATL  EV             5197                    30135   
                                      5579                    30135   
                  ORD  EV             6021                    30135   
           ABQ    ATL  DL             1784                    30140   
                                      2007                    30140   

                                                            origin_city_name  \
fl_date    origin dest unique_carrier fl_num                                   
2016-01-01 ABE    ATL  EV             5197    Allentown/Bethlehem/Easton, PA   
                                      5579    Allentown/Bethlehem/Easton, PA   
                  ORD  EV             6021    Allentown/Bethlehem/Easton, PA   
           ABQ    ATL  DL             1784                   Albuquerque, NM   
                                      2007                   Albuquerque, NM   

                                             origin_state_abr origin_state_nm  \
fl_date    origin dest unique_carrier fl_num                                    
2016-01-01 ABE    ATL  EV             5197                 PA    Pennsylvania   
                                      5579                 PA    Pennsylvania   
                  ORD  EV             6021                 PA    Pennsylvania   
           ABQ    ATL  DL             1784                 NM      New Mexico   
                                      2007                 NM      New Mexico   

                                              dest_airport_id  \
fl_date    origin dest unique_carrier fl_num                    
2016-01-01 ABE    ATL  EV             5197              10397   
                                      5579              10397   
                  ORD  EV             6021              13930   
           ABQ    ATL  DL             1784              10397   
                                      2007              10397   

                                              dest_airport_seq_id  \
fl_date    origin dest unique_carrier fl_num                        
2016-01-01 ABE    ATL  EV             5197                1039705   
                                      5579                1039705   
                  ORD  EV             6021                1393004   
           ABQ    ATL  DL             1784                1039705   
                                      2007                1039705   

                                              dest_city_market_id  \
fl_date    origin dest unique_carrier fl_num                        
2016-01-01 ABE    ATL  EV             5197                  30397   
                                      5579                  30397   
                  ORD  EV             6021                  30977   
           ABQ    ATL  DL             1784                  30397   
                                      2007                  30397   

                           

In [295]:
'''Drop airport_seq_id columns as they are proxy to airport_id
    - avoids identifying airports with multiple seq_id instances as different airports
'''
location_info = location_info.drop(columns=['origin_airport_seq_id','dest_airport_seq_id'])
    
#view resulting dataframe on flight origin and destination
location_info.head()

origin_airport_id  \
fl_date    origin dest unique_carrier fl_num                      
2016-01-01 ABE    ATL  EV             5197                10135   
                                      5579                10135   
                  ORD  EV             6021                10135   
           ABQ    ATL  DL             1784                10140   
                                      2007                10140   

                                              origin_city_market_id  \
fl_date    origin dest unique_carrier fl_num                          
2016-01-01 ABE    ATL  EV             5197                    30135   
                                      5579                    30135   
                  ORD  EV             6021                    30135   
           ABQ    ATL  DL             1784                    30140   
                                      2007                    30140   

                                                            origin_city_name  \
fl_date    origin dest unique_carrier fl_num                                   
2016-01-01 ABE    ATL  EV             5197    Allentown/Bethlehem/Easton, PA   
                                      5579    Allentown/Bethlehem/Easton, PA   
                  ORD  EV             6021    Allentown/Bethlehem/Easton, PA   
           ABQ    ATL  DL             1784                   Albuquerque, NM   
                                      2007                   Albuquerque, NM   

                                             origin_state_abr origin_state_nm  \
fl_date    origin dest unique_carrier fl_num                                    
2016-01-01 ABE    ATL  EV             5197                 PA    Pennsylvania   
                                      5579                 PA    Pennsylvania   
                  ORD  EV             6021                 PA    Pennsylvania   
           ABQ    ATL  DL             1784                 NM      New Mexico   
                                      2007                 NM      New Mexico   

                                              dest_airport_id  \
fl_date    origin dest unique_carrier fl_num                    
2016-01-01 ABE    ATL  EV             5197              10397   
                                      5579              10397   
                  ORD  EV             6021              13930   
           ABQ    ATL  DL             1784              10397   
                                      2007              10397   

                                              dest_city_market_id  \
fl_date    origin dest unique_carrier fl_num                        
2016-01-01 ABE    ATL  EV             5197                  30397   
                                      5579                  30397   
                  ORD  EV             6021                  30977   
           ABQ    ATL  DL             1784                  30397   
                                      2007                  30397   

                                             dest_city_name dest_state_abr  \
fl_date    origin dest unique_carrier fl_num                                 
2016-01-01 ABE    ATL  EV             5197      Atlanta, GA             GA   
                                      5579      Atlanta, GA             GA   
                  ORD  EV             6021      Chicago, IL             IL   
           ABQ    ATL  DL             1784      Atlanta, GA             GA   
                                      2007      Atlanta, GA             GA   

                                             dest_state_nm  
fl_date    origin dest unique_carrier fl_num                
2016-01-01 ABE    ATL  EV             5197         Georgia  
                                      5579         Georgia  
                  ORD  EV             6021        Illinois  
           ABQ    ATL  DL             1784         Georgia  
                                      2007         Georgia

In [296]:
"""Remove state abbreviation suffix from city_name values
    - already present in state_abr columns
    - create function to remove abbreviations from list of columns in a dataframe
"""
def remove_abbr(dataframe, lst_of_columns):
    for col in lst_of_columns:
        dataframe[col] = dataframe[col].str[:-4]

#call function on location info for origin and destination city name columns
remove_abbr(location_info,['origin_city_name','dest_city_name'])

#view final dataframe on flight origin and destination
location_info.head()

origin_airport_id  \
fl_date    origin dest unique_carrier fl_num                      
2016-01-01 ABE    ATL  EV             5197                10135   
                                      5579                10135   
                  ORD  EV             6021                10135   
           ABQ    ATL  DL             1784                10140   
                                      2007                10140   

                                              origin_city_market_id  \
fl_date    origin dest unique_carrier fl_num                          
2016-01-01 ABE    ATL  EV             5197                    30135   
                                      5579                    30135   
                  ORD  EV             6021                    30135   
           ABQ    ATL  DL             1784                    30140   
                                      2007                    30140   

                                                        origin_city_name  \
fl_date    origin dest unique_carrier fl_num                               
2016-01-01 ABE    ATL  EV             5197    Allentown/Bethlehem/Easton   
                                      5579    Allentown/Bethlehem/Easton   
                  ORD  EV             6021    Allentown/Bethlehem/Easton   
           ABQ    ATL  DL             1784                   Albuquerque   
                                      2007                   Albuquerque   

                                             origin_state_abr origin_state_nm  \
fl_date    origin dest unique_carrier fl_num                                    
2016-01-01 ABE    ATL  EV             5197                 PA    Pennsylvania   
                                      5579                 PA    Pennsylvania   
                  ORD  EV             6021                 PA    Pennsylvania   
           ABQ    ATL  DL             1784                 NM      New Mexico   
                                      2007                 NM      New Mexico   

                                              dest_airport_id  \
fl_date    origin dest unique_carrier fl_num                    
2016-01-01 ABE    ATL  EV             5197              10397   
                                      5579              10397   
                  ORD  EV             6021              13930   
           ABQ    ATL  DL             1784              10397   
                                      2007              10397   

                                              dest_city_market_id  \
fl_date    origin dest unique_carrier fl_num                        
2016-01-01 ABE    ATL  EV             5197                  30397   
                                      5579                  30397   
                  ORD  EV             6021                  30977   
           ABQ    ATL  DL             1784                  30397   
                                      2007                  30397   

                                             dest_city_name dest_state_abr  \
fl_date    origin dest unique_carrier fl_num                                 
2016-01-01 ABE    ATL  EV             5197          Atlanta             GA   
                                      5579          Atlanta             GA   
                  ORD  EV             6021          Chicago             IL   
           ABQ    ATL  DL             1784          Atlanta             GA   
                                      2007          Atlanta             GA   

                                             dest_state_nm  
fl_date    origin dest unique_carrier fl_num                
2016-01-01 ABE    ATL  EV             5197         Georgia  
                                      5579         Georgia  
                  ORD  EV             6021        Illinois  
           ABQ    ATL  DL             1784         Georgia  
                                      2007         Georgia

### Metrics

#### Bureau of Transportation Statistics Definitions Cont'd

Status of a Flight can be one of the following:

    Delayed: Flight will depart 15 or more minutes after its scheduled departure time.

    Departed: Flight has left the departure gate but may not be airborne yet.

    In Air: Flight is airborne. Takeoff time is actual takeoff or "wheels up" time. The arrival time is estimated. 

    Expected: A FlightView data source indicates flight is expected to arrive at arrival airport.

    Diverted: Flight has been diverted from its scheduled destination to a different location.

    Recovery: Flight had departed the diverted location and enroute or landed at the scheduled destination.

    Landed: Flight has landed. The landing time is actual touchdown or "wheels down."

    Arrived: Flight has arrived at its destination gate.

    Cancelled: Flight has been cancelled.


CrsDepTime 
    - Course Departure Time is the scheduled time, in 24-hr notation, at which the flight is meant to enter 'Departed' status
DepTime
    - Departure Time is the actual time, in 24-hr notation, at which the flight entered 'Departed' status
DepDelay
    - Departure Delay is the deviation, in minutes, of DepTime from CrsDepTime
DepDelayNew
    - Departure Delay New is the deviation, in minutes, of DepTime from CrsDepTime, accounting only for instances of late departure i.e. positive values in DepDelay
WheelsOn
    - The time, in 24-hr notation, at which the flight has gone from 'In Air' status to 'Landed' status
TaxiIn
    - Time, in minutes, which flight spends in 'Landed' status before arriving at destination gate and reaching 'Arrived' status
CrsArrTime
    - Course Arrival Time is the scheduled time, in 24-hr notation, at which the flight is meant to enter 'Arrived' status
ArrTime
    - Arrival Time is the actual time, in 24-hr notation, at which the flight entered 'Arrived' status
ArrDelay
    - Arrival Delay is the deviation, in minutes, of ArrTime from CrsArrTime
ArrDelayNew
    - Arrival Delay New is the deviation, in minutes, of ArrTime from CrsArrTime, accounting only for instances of late arrival i.e. positive values in ArrDelay
CrsElapsedTime
    - Scheduled duration of flight, from 'Departed' status to 'Arrived' status, in minutes
ActualElapsedTime
    - Actual duration of flight, from 'Departed' status to 'Arrived' status, in minutes
AirTime
    - Time, in minutes, which flight spends in 'In Air' status
Flights
    - number of flights accounted for in row
Distance
    - Distance, in miles, between origin airport location and destination airport location

In [297]:
#view unique values in flight metrics columns
print(uv_dict(flight_metrics))

#view columns pertaining to flight metrics
flight_metrics.head()

{'crs_dep_time': array([   1,    2,    3, ..., 2357, 2358, 2359], dtype=int64), 'dep_time': array([1.000e+00, 2.000e+00, 3.000e+00, ..., 2.358e+03, 2.359e+03,
       2.400e+03]), 'dep_delay': array([ -60.,  -58.,  -54., ..., 1663., 1964., 2040.]), 'dep_delay_new': array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.663e+03, 1.964e+03,
       2.040e+03]), 'wheels_on': array([1.000e+00, 2.000e+00, 3.000e+00, ..., 2.358e+03, 2.359e+03,
       2.400e+03]), 'taxi_in': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  

crs_dep_time  dep_time  \
fl_date    origin dest unique_carrier fl_num                           
2016-01-01 ABE    ATL  EV             5197            1610    1609.0   
                                      5579             630     700.0   
                  ORD  EV             6021             651     718.0   
           ABQ    ATL  DL             1784            1315    1341.0   
                                      2007             800     757.0   

                                              dep_delay  dep_delay_new  \
fl_date    origin dest unique_carrier fl_num                             
2016-01-01 ABE    ATL  EV             5197         -1.0            0.0   
                                      5579         30.0           30.0   
                  ORD  EV             6021         27.0           27.0   
           ABQ    ATL  DL             1784         26.0           26.0   
                                      2007         -3.0            0.0   

                                              wheels_on  taxi_in  \
fl_date    origin dest unique_carrier fl_num                       
2016-01-01 ABE    ATL  EV             5197       1812.0      5.0   
                                      5579        900.0      7.0   
                  ORD  EV             6021        824.0      8.0   
           ABQ    ATL  DL             1784       1813.0      6.0   
                                      2007       1230.0      4.0   

                                              crs_arr_time  arr_time  \
fl_date    origin dest unique_carrier fl_num                           
2016-01-01 ABE    ATL  EV             5197            1828    1817.0   
                                      5579             845     907.0   
                  ORD  EV             6021             811     832.0   
           ABQ    ATL  DL             1784            1810    1819.0   
                                      2007            1258    1234.0   

                                              arr_delay  arr_delay_new  \
fl_date    origin dest unique_carrier fl_num                             
2016-01-01 ABE    ATL  EV             5197        -11.0            0.0   
                                      5579         22.0           22.0   
                  ORD  EV             6021         21.0           21.0   
           ABQ    ATL  DL             1784          9.0            9.0   
                                      2007        -24.0            0.0   

                                              crs_elapsed_time  \
fl_date    origin dest unique_carrier fl_num                     
2016-01-01 ABE    ATL  EV             5197               138.0   
                                      5579               135.0   
                  ORD  EV             6021               140.0   
           ABQ    ATL  DL             1784               175.0   
                                      2007               178.0   

                                              actual_elapsed_time  air_time  \
fl_date    origin dest unique_carrier fl_num                                  
2016-01-01 ABE    ATL  EV             5197                  128.0     115.0   
                                      5579                  127.0     111.0   
                  ORD  EV             6021                  134.0     117.0   
           ABQ    ATL  DL             1784                  158.0     139.0   
                                      2007                  157.0     142.0   

                                              flights  distance  
fl_date    origin dest unique_carrier fl_num                     
2016-01-01 ABE    ATL  EV             5197        1.0     692.0  
                                      5579        1.0     692.0  
                  ORD  EV             6021        1.0     655.0  
           ABQ    ATL  DL             1784        1.0    1269.0  
                                      2007        1.0    1269.0

In [298]:
'''remove flights column from metrics dataframe
    - all rows contain only one flight i.e. column values are redundant
'''
flight_metrics = flight_metrics.drop(columns='flights')

#view resulting flight metrics dataframe
flight_metrics.head()

crs_dep_time  dep_time  \
fl_date    origin dest unique_carrier fl_num                           
2016-01-01 ABE    ATL  EV             5197            1610    1609.0   
                                      5579             630     700.0   
                  ORD  EV             6021             651     718.0   
           ABQ    ATL  DL             1784            1315    1341.0   
                                      2007             800     757.0   

                                              dep_delay  dep_delay_new  \
fl_date    origin dest unique_carrier fl_num                             
2016-01-01 ABE    ATL  EV             5197         -1.0            0.0   
                                      5579         30.0           30.0   
                  ORD  EV             6021         27.0           27.0   
           ABQ    ATL  DL             1784         26.0           26.0   
                                      2007         -3.0            0.0   

                                              wheels_on  taxi_in  \
fl_date    origin dest unique_carrier fl_num                       
2016-01-01 ABE    ATL  EV             5197       1812.0      5.0   
                                      5579        900.0      7.0   
                  ORD  EV             6021        824.0      8.0   
           ABQ    ATL  DL             1784       1813.0      6.0   
                                      2007       1230.0      4.0   

                                              crs_arr_time  arr_time  \
fl_date    origin dest unique_carrier fl_num                           
2016-01-01 ABE    ATL  EV             5197            1828    1817.0   
                                      5579             845     907.0   
                  ORD  EV             6021             811     832.0   
           ABQ    ATL  DL             1784            1810    1819.0   
                                      2007            1258    1234.0   

                                              arr_delay  arr_delay_new  \
fl_date    origin dest unique_carrier fl_num                             
2016-01-01 ABE    ATL  EV             5197        -11.0            0.0   
                                      5579         22.0           22.0   
                  ORD  EV             6021         21.0           21.0   
           ABQ    ATL  DL             1784          9.0            9.0   
                                      2007        -24.0            0.0   

                                              crs_elapsed_time  \
fl_date    origin dest unique_carrier fl_num                     
2016-01-01 ABE    ATL  EV             5197               138.0   
                                      5579               135.0   
                  ORD  EV             6021               140.0   
           ABQ    ATL  DL             1784               175.0   
                                      2007               178.0   

                                              actual_elapsed_time  air_time  \
fl_date    origin dest unique_carrier fl_num                                  
2016-01-01 ABE    ATL  EV             5197                  128.0     115.0   
                                      5579                  127.0     111.0   
                  ORD  EV             6021                  134.0     117.0   
           ABQ    ATL  DL             1784                  158.0     139.0   
                                      2007                  157.0     142.0   

                                              distance  
fl_date    origin dest unique_carrier fl_num            
2016-01-01 ABE    ATL  EV             5197       692.0  
                                      5579       692.0  
                  ORD  EV             6021       655.0  
           ABQ    ATL  DL             1784      1269.0  
                                      2007      1269.0

In [299]:
'''rename delay columns for clarity
    - dep_delay and arr_delay to dep_deviation and arr_deviation
        - as values can also indicate instances of early flights
    - dep_delay_new and arr_delay_new to dep_delay and arr_delay
        - as they are exclusively indicative of flight delay
'''
flight_metrics.rename(columns={'dep_delay':'dep_dev', 'dep_delay_new':'dep_delay', 'arr_delay':'arr_dev','arr_delay_new':'arr_delay'}, inplace=True)

#view resulting flight metrics dataframe
flight_metrics.head()

crs_dep_time  dep_time  dep_dev  \
fl_date    origin dest unique_carrier fl_num                                    
2016-01-01 ABE    ATL  EV             5197            1610    1609.0     -1.0   
                                      5579             630     700.0     30.0   
                  ORD  EV             6021             651     718.0     27.0   
           ABQ    ATL  DL             1784            1315    1341.0     26.0   
                                      2007             800     757.0     -3.0   

                                              dep_delay  wheels_on  taxi_in  \
fl_date    origin dest unique_carrier fl_num                                  
2016-01-01 ABE    ATL  EV             5197          0.0     1812.0      5.0   
                                      5579         30.0      900.0      7.0   
                  ORD  EV             6021         27.0      824.0      8.0   
           ABQ    ATL  DL             1784         26.0     1813.0      6.0   
                                      2007          0.0     1230.0      4.0   

                                              crs_arr_time  arr_time  arr_dev  \
fl_date    origin dest unique_carrier fl_num                                    
2016-01-01 ABE    ATL  EV             5197            1828    1817.0    -11.0   
                                      5579             845     907.0     22.0   
                  ORD  EV             6021             811     832.0     21.0   
           ABQ    ATL  DL             1784            1810    1819.0      9.0   
                                      2007            1258    1234.0    -24.0   

                                              arr_delay  crs_elapsed_time  \
fl_date    origin dest unique_carrier fl_num                                
2016-01-01 ABE    ATL  EV             5197          0.0             138.0   
                                      5579         22.0             135.0   
                  ORD  EV             6021         21.0             140.0   
           ABQ    ATL  DL             1784          9.0             175.0   
                                      2007          0.0             178.0   

                                              actual_elapsed_time  air_time  \
fl_date    origin dest unique_carrier fl_num                                  
2016-01-01 ABE    ATL  EV             5197                  128.0     115.0   
                                      5579                  127.0     111.0   
                  ORD  EV             6021                  134.0     117.0   
           ABQ    ATL  DL             1784                  158.0     139.0   
                                      2007                  157.0     142.0   

                                              distance  
fl_date    origin dest unique_carrier fl_num            
2016-01-01 ABE    ATL  EV             5197       692.0  
                                      5579       692.0  
                  ORD  EV             6021       655.0  
           ABQ    ATL  DL             1784      1269.0  
                                      2007      1269.0

# Step 4: Create one final flights dataframe

In [300]:
'''Merge edited content-specific dataframes into one 'Flights' dataframe'''
flights = date_info.join(location_info).join(flight_metrics)

#view final flights dataframe
print(flights.shape)
print(flights.isnull().mean())
print(flights.head())
print(flights.tail())

(1824403, 28)
quarter                  0.0
month                    0.0
day_of_month             0.0
day_of_week              0.0
origin_airport_id        0.0
origin_city_market_id    0.0
origin_city_name         0.0
origin_state_abr         0.0
origin_state_nm          0.0
dest_airport_id          0.0
dest_city_market_id      0.0
dest_city_name           0.0
dest_state_abr           0.0
dest_state_nm            0.0
crs_dep_time             0.0
dep_time                 0.0
dep_dev                  0.0
dep_delay                0.0
wheels_on                0.0
taxi_in                  0.0
crs_arr_time             0.0
arr_time                 0.0
arr_dev                  0.0
arr_delay                0.0
crs_elapsed_time         0.0
actual_elapsed_time      0.0
air_time                 0.0
distance                 0.0
dtype: float64
                                              quarter  month  day_of_month  \
fl_date    origin dest unique_carrier fl_num                                 
201